[Reference](https://medium.com/@borandabak/building-a-full-stack-machine-learning-web-application-integrating-fastapi-streamlit-80babd19c728)

In [2]:
import requests
import streamlit as st
import json

# FastAPI endpoint
API_URL = "http://localhost:8000/predict"

# Model options dictionary
models = {
    "DecisionTree": "DecisionTree",
    "KNN": "KNN",
    "LogisticRegression": "logistic_regression_model"
}

# Streamlit app
def main():
    st.title("Machine Learning Model Predictor")

    # Model selection dropdown
    selected_model = st.selectbox("Select a model", list(models.keys()))

    # Get model file name based on selection
    model_file = models[selected_model]

    # Feature inputs
    sepal_length = st.number_input("Sepal length")
    sepal_width = st.number_input("Sepal width")
    petal_length = st.number_input("Petal length")
    petal_width = st.number_input("Petal width")

    # Make prediction on button click
    if st.button("Predict"):
        # Prepare feature data as JSON payload
        feature_data = {
            "sepal_length": sepal_length,
            "sepal_width": sepal_width,
            "petal_length": petal_length,
            "petal_width": petal_width
        }
        feature_data = [sepal_length, sepal_width, petal_length, petal_width]

        # Call FastAPI endpoint and get prediction result
        headers = {'Content-Type': 'application/json'}
        response = requests.post(API_URL + f"/{model_file}", json=feature_data)

        # Display prediction result
        st.write(f"Prediction: {response.json()}")
    
if __name__ == "__main__":
    main()

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle

# Load the Iris dataset
iris_df = pd.read_csv('IRIS.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    iris_df.iloc[:, :-1], iris_df.iloc[:, -1], test_size=0.2, random_state=42)

# Train the model on the training set
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Save the trained model to a file
with open('DecisionTree.pkl', 'wb') as f:
    pickle.dump(model, f)

In [4]:
from fastapi import FastAPI
from typing import List
import joblib

app = FastAPI()


@app.post('/predict/DecisionTree')
async def predict(data: List[float]):

    model = joblib.load("models/DecisionTree.pkl")

    prediction = model.predict([data])[0]
    return prediction

@app.post('/predict/KNN')
async def predict(data: List[float]):

    model = joblib.load("models/KNN.pkl")

    prediction = model.predict([data])[0]
    return prediction

@app.post('/predict/LogisticRegression')
async def predict(data: List[float]):

    model = joblib.load("models/logistic_regression_model.pkl")

    prediction = model.predict([data])[0]
    return prediction